# A homemade stream cipher

In this notebook we will use a pseudorandom generator to build our first homemade cipher

## Encoding a message to bytes

To code our first cipher based on a pseudorandom generator we need to understand a little bit better how information is encoded in python. For this, we coded several functions to help us.

In [1]:
from crypto import bytes_to_bin, bytes_to_hex

message = b"simple message"

bin_repr = bytes_to_bin(message, pre="")
hex_repr = bytes_to_hex(message, pre="")

print(f"message:\n{message}\nlen bytes: {len(message)}\n")
print(f"message in binary:\n{bin_repr}\nlen bits: {len(bin_repr)}\n")
print(f"message in hexadecimal:\n{hex_repr}\nlen hex:{len(hex_repr)}\n")

message:
b'simple message'
len bytes: 14

message in binary:
0111001101101001011011010111000001101100011001010010000001101101011001010111001101110011011000010110011101100101
len bits: 112

message in hexadecimal:
73696d706c65206d657373616765
len hex:28



## Stream Cipher

In our first cipher we are going to pad our messeage with a pseudorandom "stream" of bytes. This kind of cipher is known as stream cipher.

<img src="img/stream_cipher.png" style="width:900px"/>

## Padding bytes (XOR operation)

We need to implement the XOR operation byte-wise, that is how we "pad" our message to convert it to its encrypted form.

In [4]:
from crypto import binary, hexadecimal
import random

a = random.randrange(256)
b = random.randrange(256)

print(f"a: {a} (int), {hexadecimal(a, pre='')} (hex), {binary(a, pre='')} (bin)")
print(f"b: {b} (int), {hexadecimal(b, pre='')} (hex), {binary(b, pre='')} (bin)")

xored = a ^ b
print(f"x: {xored} (int), {hexadecimal(xored, pre='')} (hex), {binary(xored, pre='')} (bin)")

print(f"\n{binary(a, pre='')}\n+\n{binary(b, pre='')}\n=\n{binary(xored, pre='')}")

a: 195 (int), c3 (hex), 11000011 (bin)
b: 191 (int), bf (hex), 10111111 (bin)
x: 124 (int), 7c (hex), 01111100 (bin)

11000011
+
10111111
=
01111100


## Bytes random generator

We need a pseudorandom genertor for bytes, let's code it!

In Python we have the ```state``` variable to store the state of the pseudorandom generator in ```random``` package.

In [24]:
# in random python pacakge we have the "state" of the PRG
random.seed(10)
state = random.getstate()

print([random.randrange(256) for _ in range(10)])

random.setstate(state)
print([random.randrange(256) for _ in range(10)])


[16, 219, 247, 7, 105, 236, 251, 142, 82, 17]
[16, 219, 247, 7, 105, 236, 251, 142, 82, 17]


In [25]:
def PseudoRandomBytes(state: tuple, l: int) -> (tuple, bytes):
    """
    Generates a stream of pseudorandom bytes
    Input:
        - state: a state for the python random pacakge (random.getstate())
        - l: length of the pseudorandom stream of bytes
    Returns:
        - state: the current state of the random
        - bytestream: a bytes class of lenght l
    """
    random.setstate(state)
    prng = []
    
    while len(prng) < l:
        prng.append(hexadecimal(random.randrange(256)))
    
    return random.getstate(), bytes([int(x, 0) for x in prng])

In [27]:
state = random.getstate()
l = 20

new_state, prng = PseudoRandomBytes(state, l)
print(prng)

b'\xe1\xc0\x16\x02xDc\x9b\xbbz\xa0\xe6\xdf\xf0!\xa6Pr\xd3z'


## A class for the parties, let's communicate securely!

In [30]:
class Party:
    def __init__(self, state: tuple):
        self._state = state
        
    def encrypt_decrypt(self, m: bytes) -> bytes:
        new_state, random_bytes = PseudoRandomBytes(self._state, len(m))
        self._state = new_state
        
        return bytes([a ^ b for a, b in zip(m, random_bytes)])

In [33]:
state = random.getstate()

alice = Party(state)
bob = Party(state)

In [34]:
m = b"Hi Bob. How are you doing?"
ctx = alice.encrypt_decrypt(m)
m2 = bob.encrypt_decrypt(ctx)

print(f"message:\n\t{m}\n")
print(f"ciphertext:\n\t{ctx}\n")
print(f"recovered_message:\n\t{m2}\n")

message:
	b'Hi Bob. How are you doing?'

ciphertext:
	b'\xd2=y\x18\x9a\xd3\x89\xfe:ma\x89\xc2\x0eM\xa6\x9c\xa0$\xe7\x99\x14\xe2\x99\x98\x1f'

recovered_message:
	b'Hi Bob. How are you doing?'



In [35]:
m = b"I'm good, thank you!. How do you do?"
ctx = bob.encrypt_decrypt(m)
m2 = alice.encrypt_decrypt(ctx)


print(f"message:\n\t{m}\n")
print(f"ciphertext:\n\t{ctx}\n")
print(f"recovered_message:\n\t{m2}\n")

message:
	b"I'm good, thank you!. How do you do?"

ciphertext:
	b'\x1c\xdd\x81\xed"\xb8\xdd\xa2\xd6t\x95Z\xb5|j\xc0Xv\xc6\x0ec\x17\xa0\x970\xca\xbb\x8b\x8f\xfd\x85\xad\x88*\x16\x9a'

recovered_message:
	b"I'm good, thank you!. How do you do?"

